In [2]:
# EDA
import pandas as pd
import numpy as np

# Data Preprocessing
from sklearn import preprocessing

# Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Recommender System Imps
# Content Based Filtering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Collaborative Based Filtering
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# To work with text data
import re
import string

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Food Recommendation System/food dataset.csv')
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,"white balsamic vinegar, lemon juice, lemon rin..."
1,2,chicken minced salad,Healthy Food,non-veg,"olive oil, chicken mince, garlic (minced), oni..."
2,3,sweet chilli almonds,Snack,veg,"almonds whole, egg white, curry leaves, salt, ..."
3,4,tricolour salad,Healthy Food,veg,"vinegar, honey/sugar, soy sauce, salt, garlic ..."
4,5,christmas cake,Dessert,veg,"christmas dry fruits (pre-soaked), orange zest..."


In [4]:
# เมนูอาหารใน dataset
len(list(df['Name'].unique()))

400

In [5]:
# ประเภทของอาหาร
df['C_Type'].unique()

array(['Healthy Food', 'Snack', 'Dessert', 'Japanese', 'Indian', 'French',
       'Mexican', 'Italian', 'Chinese', 'Beverage', 'Thai', 'Korean',
       ' Korean', 'Vietnames', 'Nepalese', 'Spanish'], dtype=object)

In [6]:
# อาหารมังสวิรัติ / ไม่มังสวิรัติ
df['Veg_Non'].unique()

array(['veg', 'non-veg'], dtype=object)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Food_ID   400 non-null    int64 
 1   Name      400 non-null    object
 2   C_Type    400 non-null    object
 3   Veg_Non   400 non-null    object
 4   Describe  400 non-null    object
dtypes: int64(1), object(4)
memory usage: 15.8+ KB


In [8]:
# ลบเครื่องหมายวรรคตอน ทั้งหมดออกจากข้อความในคอลัมน์ Describe
def text_cleaning(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    return text

In [9]:
df['Describe'] = df['Describe'].apply(text_cleaning)

In [10]:
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe
0,1,summer squash salad,Healthy Food,veg,white balsamic vinegar lemon juice lemon rind ...
1,2,chicken minced salad,Healthy Food,non-veg,olive oil chicken mince garlic minced onion sa...
2,3,sweet chilli almonds,Snack,veg,almonds whole egg white curry leaves salt suga...
3,4,tricolour salad,Healthy Food,veg,vinegar honeysugar soy sauce salt garlic clove...
4,5,christmas cake,Dessert,veg,christmas dry fruits presoaked orange zest lem...


# Content Based Filtering

Simple Content Based Filtering

TF-IDF ใช้ปรับค่าความถี่ (count) ของคำในเอกสารให้อยู่ในรูปของค่าทศนิยม (floating-point value) เพื่อให้เหมาะสมกับการใช้งานร่วมกับตัวจำแนก (classifier)

In [11]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Describe'])
tfidf_matrix.shape

(400, 1261)

Linear Kernel เป็นฟังก์ชันที่ใช้ในการคำนวณความคล้ายคลึง (similarity) ระหว่างเวกเตอร์สองตัวในรูปแบบเชิงเส้น โดย Linear Kernel เป็นกรณีพิเศษของ Polynomial Kernel ที่มีค่าพารามิเตอร์ degree = 1 และ coef0 = 0 ซึ่งเรียกว่า homogeneous kernel (ไม่มีค่าคงที่เพิ่มเข้ามา)

In [12]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.16228366, 0.13001124, ..., 0.1286286 , 0.04277223,
        0.09993639],
       [0.16228366, 1.        , 0.06799336, ..., 0.14878001, 0.05688681,
        0.16917639],
       [0.13001124, 0.06799336, 1.        , ..., 0.03291577, 0.11795401,
        0.01834168],
       ...,
       [0.1286286 , 0.14878001, 0.03291577, ..., 1.        , 0.        ,
        0.10087579],
       [0.04277223, 0.05688681, 0.11795401, ..., 0.        , 1.        ,
        0.        ],
       [0.09993639, 0.16917639, 0.01834168, ..., 0.10087579, 0.        ,
        1.        ]])

In [13]:
# ใช้ชื่อเมนูอาหาร
indices = pd.Series(df.index, index=df['Name']).drop_duplicates()
indices

,0
Name,
summer squash salad,0
chicken minced salad,1
sweet chilli almonds,2
tricolour salad,3
christmas cake,4
...,...
Kimchi Toast,395
"Tacos de Gobernador (Shrimp, Poblano, and Cheese Tacos)",396
Melted Broccoli Pasta With Capers and Anchovies,397


In [14]:
def get_recommendations(title, cosine_sim=cosine_sim):

    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # จะได้คะแนนของอาหาร 5 รายการที่มีความคล้ายคลึงกันมากที่สุด
    sim_scores = sim_scores[1:6]

    food_indices = [i[0] for i in sim_scores]
    return df['Name'].iloc[food_indices]

Advanced Content Based Filtering

In [15]:
# รวม features ที่จะช่วยให้ระบบแนะนำแม่นยำมากขึ้น (ประเภทของอาหาร, อาหารมังสวิรัติ / ไม่มังสวิรัติ, วัตถุดิบของอาหาร)
features = ['C_Type','Veg_Non', 'Describe']

Soup คือ การผสมผสานองค์ประกอบต่าง ๆ ดังนั้นจึงมีการสร้างคอลัมน์ใหม่ขึ้นมา เพื่อรวบรวมคุณสมบัติสำคัญทั้งหมดเข้าไว้ด้วยกันในรูปแบบข้อความ

In [16]:
def create_soup(x):
    return x['C_Type'] + " " + x['Veg_Non'] + " " + x['Describe']

In [17]:
df['soup'] = df.apply(create_soup, axis=1)

In [18]:
df.head()

,Food_ID,Name,C_Type,Veg_Non,Describe,soup
0,1,summer squash salad,Healthy Food,veg,white balsamic vinegar lemon juice lemon rind ...,Healthy Food veg white balsamic vinegar lemon ...
1,2,chicken minced salad,Healthy Food,non-veg,olive oil chicken mince garlic minced onion sa...,Healthy Food non-veg olive oil chicken mince g...
2,3,sweet chilli almonds,Snack,veg,almonds whole egg white curry leaves salt suga...,Snack veg almonds whole egg white curry leaves...
3,4,tricolour salad,Healthy Food,veg,vinegar honeysugar soy sauce salt garlic clove...,Healthy Food veg vinegar honeysugar soy sauce ...
4,5,christmas cake,Dessert,veg,christmas dry fruits presoaked orange zest lem...,Dessert veg christmas dry fruits presoaked ora...


Count Vectorizer เป็นเทคนิคที่ใช้แปลงชุดเอกสารข้อความ (text documents) ให้กลายเป็นเมทริกซ์ที่แสดงจำนวนครั้ง (counts) ของแต่ละคำ (tokens) ที่ปรากฏในเอกสารนั้น ๆ โดยผลลัพธ์จะอยู่ในรูปแบบของ sparse matrix ซึ่งมีประสิทธิภาพในการจัดเก็บข้อมูลที่มีค่าศูนย์จำนวนมาก (จัดการโดย scipy.sparse.csr_matrix)

fit_transform เป็นการ
- เรียนรู้คำศัพท์ (Learn the vocabulary dictionary):
สร้างรายการคำทั้งหมด (vocabulary) ที่พบในเอกสารข้อความ โดยแต่ละคำจะถูกกำหนดดัชนี (index) สำหรับการใช้งานในเมทริกซ์
- แปลงเอกสารข้อความเป็น Document-Term Matrix:
สร้างเมทริกซ์ที่แสดงจำนวนครั้งที่แต่ละคำปรากฏในเอกสารแต่ละฉบับ

In [19]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

Cosine Similarity คือฟังก์ชันที่ใช้คำนวณ "ความคล้ายคลึงเชิงมุม" (angular similarity) ระหว่างตัวอย่างสองตัว (samples) ในเวกเตอร์
𝑋 และ 𝑌 โดยจะวัดความคล้ายกันตามมุมระหว่างเวกเตอร์ในพื้นที่หลายมิติ (multi-dimensional space) โดยใช้การคำนวณความคล้ายกันของข้อความ

In [20]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [21]:
# รีเซ็ตค่า index ของ DataFrame และสร้าง Series ใหม่ที่ประกอบด้วยชื่ออาหาร (Name) เป็นดัชนี (index) และค่าของ Series เป็นตำแหน่ง (index) เดิมของ DataFrame
df = df.reset_index()
indices = pd.Series(df.index, index=df['Name'])

In [22]:
display(indices)

,0
Name,
summer squash salad,0
chicken minced salad,1
sweet chilli almonds,2
tricolour salad,3
christmas cake,4
...,...
Kimchi Toast,395
"Tacos de Gobernador (Shrimp, Poblano, and Cheese Tacos)",396
Melted Broccoli Pasta With Capers and Anchovies,397


Testing Content Based Filtering

In [23]:
# First Model - Simple Content Based Filtering
get_recommendations('meat lovers pizza')

,Name
154,chilli fish
399,Rosemary Roasted Vegetables
258,ragi coconut ladoo (laddu)
124,cheese chicken kebabs
263,kuttu atta pizza


In [24]:
# Second Model - Advanced Content Based Filtering
get_recommendations('meat lovers pizza', cosine_sim2)

,Name
124,cheese chicken kebabs
286,pasta in cheese sauce
154,chilli fish
18,baked shankarpali
101,dahi chicken


# Collaborative Filtering

In [ ]:
# อ่านไฟล์ ratings
rating = pd.read_csv('/content/drive/MyDrive/Food Recommendation System/ratings.csv')
rating.head()

,User_ID,Food_ID,Rating
0,1,88,4
1,1,46,3
2,1,24,5
3,1,25,4
4,2,49,1


In [ ]:
rating.shape

(511, 3)

In [ ]:
rating.isnull().sum()

,0
User_ID,0
Food_ID,0
Rating,0


In [ ]:
# สร้าง DataFrame ใหม่ที่มี Food_ID และจำนวนการให้คะแนน (Rating_count) สำหรับแต่ละอาหาร
food_rating = rating.groupby(by = 'Food_ID').count()
food_rating = food_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
food_rating

,Food_ID,Rating_count
0,1,2
1,2,3
2,3,2
3,4,2
4,5,6
...,...,...
304,305,1
305,306,1
306,307,1
307,308,1


In [ ]:
# สร้าง DataFrame ใหม่ที่มี User_ID และจำนวนการให้คะแนน (Rating_count) สำหรับแต่ละผู้ใช้
user_rating = rating.groupby(by='User_ID').count()
user_rating = user_rating['Rating'].reset_index().rename(columns={'Rating':'Rating_count'})
user_rating

,User_ID,Rating_count
0,1,4
1,2,4
2,3,9
3,4,6
4,5,6
...,...,...
95,96,6
96,97,7
97,98,7
98,99,6


In [ ]:
# ใช้ฟังก์ชัน pivot_table เพื่อจัดข้อมูลใหม่ในรูปแบบเมทริกซ์ที่แสดงการให้คะแนน (Rating) ของผู้ใช้ (User_ID) สำหรับแต่ละอาหาร (Food_ID)
rating_matrix = rating.pivot_table(index='Food_ID',columns='User_ID',values='Rating').fillna(0)
rating_matrix.head()

User_ID,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
Food_ID,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,7.0,0.0,0.0


In [ ]:
rating_matrix.shape

(309, 100)

sparse matrix ใช้สำหรับการคำนวณที่ต้องใช้ข้อมูลจากคอลัมน์หลายๆ ตัว แต่ไม่ต้องการเก็บค่า 0

In [ ]:
csr_rating_matrix =  csr_matrix(rating_matrix.values)
print(csr_rating_matrix)

  (0, 48)	5.0
  (0, 70)	10.0
  (1, 8)	3.0
  (1, 21)	5.0
  (1, 38)	10.0
  (2, 76)	1.0
  (2, 88)	7.0
  (3, 40)	6.0
  (3, 89)	6.0
  (4, 27)	10.0
  (4, 38)	10.0
  (4, 72)	7.0
  (4, 87)	3.0
  (4, 93)	2.0
  (4, 97)	7.0
  (5, 42)	10.0
  (5, 48)	6.0
  (5, 76)	6.0
  (5, 91)	1.0
  (6, 16)	4.0
  (6, 47)	5.0
  (6, 62)	9.0
  (6, 69)	8.0
  (6, 70)	8.0
  (7, 4)	6.0
  :	:
  (284, 30)	9.0
  (285, 80)	6.0
  (286, 24)	3.0
  (287, 54)	3.0
  (288, 55)	9.0
  (289, 31)	7.0
  (290, 15)	1.0
  (291, 2)	8.0
  (292, 95)	5.0
  (293, 41)	4.0
  (294, 43)	10.0
  (295, 41)	10.0
  (296, 94)	5.0
  (297, 55)	4.0
  (298, 2)	1.0
  (299, 28)	9.0
  (300, 53)	1.0
  (301, 77)	5.0
  (302, 63)	6.0
  (303, 29)	1.0
  (304, 55)	9.0
  (305, 79)	8.0
  (306, 70)	1.0
  (307, 96)	3.0
  (308, 31)	5.0


In [ ]:
# ใช้ Cosine Similarity เพื่อหา(similarity) และแนะนำ(nearest neighbors) ในข้อมูลการให้คะแนน (rating) โดยใช้ฟังก์ชัน NearestNeighbors จาก Scikit-learn
recommender = NearestNeighbors(metric='cosine')
recommender.fit(csr_rating_matrix)

NearestNeighbors(metric='cosine')

In [ ]:
def Get_Recommendations(title):
    user= df[df['Name']==title]
    user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]
    user_ratings = rating_matrix.iloc[user_index]

    reshaped = user_ratings.values.reshape(1,-1)
    distances, indices = recommender.kneighbors(reshaped,n_neighbors=16)

    nearest_neighbors_indices = rating_matrix.iloc[indices[0]].index[1:]
    nearest_neighbors = pd.DataFrame({'Food_ID': nearest_neighbors_indices})

    result = pd.merge(nearest_neighbors,df,on='Food_ID',how='left')

    return result.head()

Testing Collaborative Filtering

In [ ]:
# Third Model - Collaborative Filtering
Get_Recommendations('meat lovers pizza')

<ipython-input-53-41575ee51f13>:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  user_index = np.where(rating_matrix.index==int(user['Food_ID']))[0][0]


,Food_ID,index,Name,C_Type,Veg_Non,Describe,soup
0,202,201,pista chocolate & mandarin,Dessert,veg,Pistachios milk sugar broken rice green cardam...,Dessert veg Pistachios milk sugar broken rice ...
1,238,237,holi special malai kofta,Indian,veg,potatoes paneer cottage cheese maida coriander...,Indian veg potatoes paneer cottage cheese maid...
2,219,218,mother christmas cake,Dessert,veg,tart apples 2 large sugar apple juice eggs veg...,Dessert veg tart apples 2 large sugar apple ju...
3,179,178,shiitake fried rice with water chestnuts,Chinese,veg,Shitake Mushrooms Vegetable Oil Garlic Buds Gr...,Chinese veg Shitake Mushrooms Vegetable Oil Ga...
4,142,141,fish skewers with coriander and red wine vineg...,Thai,non-veg,sea bass fillets olive oil for grilling red wi...,Thai non-veg sea bass fillets olive oil for gr...
